In [1]:
# Check GPU device.
from keras import backend as K
print(K.tensorflow_backend._get_available_gpus())

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1']


In [2]:
#https://richliao.github.io/supervised/classification/2016/11/26/textclassifier-convolutional/
#https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
#RNN

import os
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool
from spellchecker import SpellChecker
import string
import math

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
from sklearn.model_selection import train_test_split
from nltk import word_tokenize

# For encoding labels.
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

## Read and compile tranining and validation dataset.

In [3]:
train_file_path='../../dataset/df_ntee_universal/train/'
file_list=os.listdir(train_file_path)
df_train=pd.DataFrame()
for file in file_list:
    df_train=pd.concat([df_train, pd.read_pickle(train_file_path+file, compression='gzip')])

test_file_path='../../dataset/df_ntee_universal/test/'
file_list=os.listdir(test_file_path)
df_test=pd.DataFrame()
for file in file_list:
    df_test=pd.concat([df_test, pd.read_pickle(test_file_path+file, compression='gzip')])
    
len(df_train), len(df_test)

(154424, 38607)

In [4]:
df_train['mission_prgrm_spellchk']=df_train['TAXPAYER_NAME']+' '+df_train['mission_spellchk']+' '+df_train['prgrm_dsc_spellchk'] # Using spell-checked.
print(len(df_train['mission_prgrm_spellchk']), len(df_train['NTEE1'].drop_duplicates()))

df_test['mission_prgrm_spellchk']=df_test['TAXPAYER_NAME']+' '+df_test['mission_spellchk']+' '+df_test['prgrm_dsc_spellchk'] # Using spell-checked.
print(len(df_test['mission_prgrm_spellchk']), len(df_test['NTEE1'].drop_duplicates()))

154424 25
38607 25


In [5]:
# # Build training and testing data frame.
# small_num=0
# while small_num<500: # Make sure each category has at least 500 records.
#     sampleDF = df_train[df_train.mission.notna() & df_train.NTEE1.notna()].sample(120000)
#     trainDF, valDF =train_test_split(sampleDF, test_size=.3)
#     small_num=trainDF.groupby('NTEE_M').count().sort_values('EIN').iloc[0]['EIN']

# See the composition by NTEE major groups.
print(df_train.groupby('NTEE1').count()['EIN']/len(df_train), '\n'*2, df_test.groupby('NTEE1').count()['EIN']/len(df_test))

NTEE1
A    0.110151
B    0.167247
C    0.021519
D    0.027450
E    0.058378
F    0.014901
G    0.032722
H    0.003024
I    0.019084
J    0.030902
K    0.013010
L    0.038478
M    0.030390
N    0.100114
O    0.011209
P    0.059447
Q    0.012867
R    0.006890
S    0.093632
T    0.013159
U    0.006476
V    0.002266
W    0.054117
X    0.029568
Y    0.042998
Name: EIN, dtype: float64 

 NTEE1
A    0.111146
B    0.166265
C    0.021421
D    0.026783
E    0.059756
F    0.014065
G    0.035045
H    0.003264
I    0.019168
J    0.029321
K    0.013521
L    0.039811
M    0.029528
N    0.101666
O    0.010594
P    0.060041
Q    0.011293
R    0.006657
S    0.093325
T    0.014013
U    0.005828
V    0.002202
W    0.052788
X    0.028440
Y    0.044059
Name: EIN, dtype: float64


### Prepare labels.

In [7]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(list(df_train.NTEE1.unique()))

y_train=lb.transform(df_train['NTEE1'])
# y_test=lb.transform(df_test['NTEE1']) # No need to transform Y for testing.

### Prepare input text.

In [8]:
stop_list=stopwords.words('english')+list(string.punctuation)
def stopwords_remove(string):
    global stop_list
    tokens=word_tokenize(string)
    return [s for s in tokens if s not in stop_list]

In [ ]:
text_token_list_train=df_train['mission_prgrm_spellchk'].apply(stopwords_remove)
text_token_list_test=df_test['mission_prgrm_spellchk'].apply(stopwords_remove)

In [ ]:
# Build word index for train and validation texts.
tokenizer=Tokenizer()
tokenizer.fit_on_texts(text_token_list_train.to_list()+text_token_list_test.to_list())
print(list(tokenizer.word_index.items())[0:5])

In [ ]:
seq_encoding_text_train=tokenizer.texts_to_sequences(text_token_list_train)
seq_encoding_text_test=tokenizer.texts_to_sequences(text_token_list_test)

In [ ]:
# Pads sequences to the same length.
x_train=pad_sequences(sequences=seq_encoding_text_train,
                      maxlen=max([len(s) for s in seq_encoding_text_train]), # Max length of the sequence.
                      dtype = "int32", padding = "post", truncating = "post", 
                      value = 0 # Zero is used for representing None or Unknown.
                     )
x_test=pad_sequences(sequences=seq_encoding_text_test,
                    maxlen=max([len(s) for s in seq_encoding_text_train]), # Max length of the sequence.
                    dtype = "int32", padding = "post", truncating = "post", 
                    value = 0 # Zero is used for representing None or Unknown.
                     )

### Prepare embedding layer.
Use pre-trained GloVe.

In [ ]:
import gensim.downloader as api
EMBEDDING_DIM=100
glove_word_vector=api.load('glove-wiki-gigaword-'+str(EMBEDDING_DIM))

In [ ]:
embedding_matrix = np.zeros((len(tokenizer.word_index)+1, EMBEDDING_DIM))

In [ ]:
for word, index in tqdm(tokenizer.word_index.items()):
    try:
        embedding_matrix[index] = glove_word_vector.get_vector(word)
    except:
        pass
        # words not found in embedding index will be all-zeros.

In [ ]:
# https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
embedding_layer = Embedding(input_dim=len(tokenizer.word_index)+1, # Size of vocabulary.
                            input_length=max([len(s) for s in seq_encoding_text_train]), # Length of input, i.e., length of padded sequence.
                            output_dim=EMBEDDING_DIM, # Size of the vector space in which words will be embedded.
                            weights=[embedding_matrix],
                            trainable=False
                           )

### Stochastic tuning of training params.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalMaxPool1D, Conv1D
from keras.layers.advanced_activations import LeakyReLU, PReLU
import tensorflow as tf

with tf.device('/gpu:1'): # Specify which GPU to use.
    # define the model
    model = Sequential()
    model.add(embedding_layer)
    # model.add(Flatten())
    model.add(Conv1D(128, 5, activation='softplus'))
    model.add(GlobalMaxPool1D())
    model.add(Dense(units=32, activation='sigmoid'))
    model.add(Dense(units=16, activation='softplus'))
    # model.add(PReLU()) # https://medium.com/tinymind/a-practical-guide-to-relu-b83ca804f1f7
    model.add(Dense(units=16, activation='tanh'))
    model.add(Dense(units=16, activation='softplus'))
    model.add(Dense(units=len(y_train[0]), activation='softmax'))
    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc', 
    #                                                                      precision, recall
                                                                        ])
    # summarize the model
    print(model.summary())

    # fit the model
    history=model.fit(x_train, y_train, validation_split=0.3, epochs=20, verbose=1)

### Try Grid Search.

In [17]:
# Continue previous work.
df_history=pd.read_csv('../../output/grid_search_history_major_group.tsv', sep='\t', index_col=0)

In [18]:
param_list_done=set(map(tuple, 
                        df_history[['conv_num_filters', 'conv_kernel_size', 'conv_act', 'out_act']].values.tolist()
                       )
                   )

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalMaxPool1D, Conv1D
import tensorflow as tf
from datetime import datetime

# df_history=pd.DataFrame()
for num_filters in [32, 64, 128]:
    for kernel_size in [3,5,7]:
        for conv_act in ['sigmoid', 'softplus', 'tanh', 'softmax']:
            for out_act in ['sigmoid', 'softplus', 'tanh', 'softmax']:
                param=tuple((num_filters, kernel_size, conv_act, out_act))
                if param not in param_list_done:
                    t1=datetime.now()
                    # Run NN on a specified GPU.
                    with tf.device('/device:GPU:1'):
                        # define the model
                        model = Sequential()
                        model.add(embedding_layer)
                        # model.add(Flatten())
                        model.add(Conv1D(num_filters, kernel_size, activation=conv_act))
                        model.add(GlobalMaxPool1D())
                        model.add(Dense(units=32, activation='sigmoid'))
                        model.add(Dense(units=32, activation='softplus'))
                        model.add(Dense(units=16, activation='tanh'))
                        model.add(Dense(units=16, activation='softplus'))
                        model.add(Dense(units=len(y_train[0]), activation=out_act))
                        # compile the model
                        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
                        # F1, precision, and recall removed. https://github.com/keras-team/keras/issues/5794
                        # fit the model
                        history=model.fit(x_train, y_train, validation_split=0.2, epochs=50, verbose=0)
                        y_prob = model.predict(x_val, verbose=0)
                    # Save history.
                    acc = history.history['acc']
                    val_acc = history.history['val_acc']
                    loss = history.history['loss']
                    val_loss = history.history['val_loss']
                    epochs = range(1, len(acc) + 1)
                    # Calculate on validation dataset.
                    y_classes = y_prob.argmax(axis=-1)
                    y_classes_prob=[s.max() for s in y_prob]
                    y_classes_val=y_val.argmax(axis=-1)
                    df_val=pd.DataFrame({'pred':y_classes, 
                                         'true':y_classes_val, 
                                         'prob':y_classes_prob})
                    val_acc_real=len(df_val[df_val.pred==df_val.true])/len(df_val)
                    # Save history to datafame.
                    df_history_temp=pd.DataFrame()
                    df_history_temp['acc']=acc
                    df_history_temp['val_acc']=val_acc
                    df_history_temp['val_acc_real']=[math.nan]*(len(epochs)-1)+[val_acc_real]
                    df_history_temp['loss']=loss
                    df_history_temp['val_loss']=val_loss
                    df_history_temp['epochs']=epochs
                    df_history_temp['conv_num_filters']=[num_filters]*len(epochs)
                    df_history_temp['conv_kernel_size']=[kernel_size]*len(epochs)
                    df_history_temp['conv_act']=[conv_act]*len(epochs)
                    df_history_temp['out_act']=[out_act]*len(epochs)
                    df_history_temp['time_stamp']=[str(t1)]+[math.nan]*(len(epochs)-2)+[str(datetime.now())]
                    df_history=df_history.append(df_history_temp, ignore_index=True)
                    df_history.to_csv('../../output/grid_search_history_major_group.tsv', sep='\t')

### Decision making: Optimizing.

In [17]:
# Continue previous work.
try:
    df_history=pd.read_csv('../../output/grid_search_history_major_group_optimizing.tsv', sep='\t', index_col=0)
except:
    df_history=pd.DataFrame(columns=pd.read_csv('../../output/grid_search_history_major_group.tsv', sep='\t', index_col=0).columns)

In [ ]:
param_list_done=set(map(tuple, 
                        df_history[['conv_num_filters', 'conv_kernel_size', 'conv_act', 'out_act']].values.tolist()
                       )
                   )

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalMaxPool1D, Conv1D
import tensorflow as tf
from datetime import datetime

# df_history=pd.DataFrame()
for num_filters in [128, 256, 512, 1024]:
    for kernel_size in [3]:
        for conv_act in ['softplus']:
            for out_act in ['sigmoid', 'softplus', 'softmax']:
                param=tuple((num_filters, kernel_size, conv_act, out_act))
                if param not in param_list_done:
                    t1=datetime.now()
                    # Run NN on a specified GPU.
                    with tf.device('/device:GPU:1'):
                        # define the model
                        model = Sequential()
                        model.add(embedding_layer)
                        # model.add(Flatten())
                        model.add(Conv1D(num_filters, kernel_size, activation=conv_act))
                        model.add(GlobalMaxPool1D())
                        model.add(Dense(units=32, activation='sigmoid'))
                        model.add(Dense(units=32, activation='softplus'))
                        model.add(Dense(units=16, activation='tanh'))
                        model.add(Dense(units=16, activation='softplus'))
                        model.add(Dense(units=len(y_train[0]), activation=out_act))
                        # compile the model
                        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
                        # F1, precision, and recall removed. https://github.com/keras-team/keras/issues/5794
                        # fit the model
                        history=model.fit(x_train, y_train, validation_split=0.2, epochs=50, verbose=0)
                        y_prob = model.predict(x_val, verbose=0)
                    # Save history.
                    acc = history.history['acc']
                    val_acc = history.history['val_acc']
                    loss = history.history['loss']
                    val_loss = history.history['val_loss']
                    epochs = range(1, len(acc) + 1)
                    # Calculate on validation dataset.
                    y_classes = y_prob.argmax(axis=-1)
                    y_classes_prob=[s.max() for s in y_prob]
                    y_classes_val=y_val.argmax(axis=-1)
                    df_val=pd.DataFrame({'pred':y_classes, 
                                         'true':y_classes_val, 
                                         'prob':y_classes_prob})
                    val_acc_real=len(df_val[df_val.pred==df_val.true])/len(df_val)
                    # Save history to datafame.
                    df_history_temp=pd.DataFrame()
                    df_history_temp['acc']=acc
                    df_history_temp['val_acc']=val_acc
                    df_history_temp['val_acc_real']=[math.nan]*(len(epochs)-1)+[val_acc_real]
                    df_history_temp['loss']=loss
                    df_history_temp['val_loss']=val_loss
                    df_history_temp['epochs']=epochs
                    df_history_temp['conv_num_filters']=[num_filters]*len(epochs)
                    df_history_temp['conv_kernel_size']=[kernel_size]*len(epochs)
                    df_history_temp['conv_act']=[conv_act]*len(epochs)
                    df_history_temp['out_act']=[out_act]*len(epochs)
                    df_history_temp['time_stamp']=[str(t1)]+[math.nan]*(len(epochs)-2)+[str(datetime.now())]
                    df_history=df_history.append(df_history_temp, ignore_index=True)
                    df_history.to_csv('../../output/grid_search_history_major_group_optimizing.tsv', sep='\t')

### Train model finalist

**Best configuration**

_Broad Category_

|acc | val_acc | val_acc_real | loss | val_loss | epochs | conv_num_filters | conv_kernel_size | conv_act | out_act|
|--|--|--|--|--|--|--|--|--|--|
|0.820386905 | 0.776488095 | -- | 0.613613255 | 0.738004138 | 4 | 512 | 3 | softplus | softplus|

_Major Group_

|acc | val_acc | val_acc_real | loss | val_loss | epochs | conv_num_filters | conv_kernel_size | conv_act | out_act|
|--|--|--|--|--|--|--|--|--|--|
|0.764369048 | 0.710428571 | -- | 0.888776311 | 1.120441045 | 6 | 256 | 3 | softplus | softmax|
|0.779 | 0.71 | -- | 0.834419103 | 1.140895644 | 7 | 256 | 3 | softplus | softplus|

Use `softplus`+`softplus`.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalMaxPool1D, Conv1D
import tensorflow as tf
from datetime import datetime

with tf.device('/device:GPU:1'):
    # define the model
    model = Sequential()
    model.add(embedding_layer)
    # model.add(Flatten())
    model.add(Conv1D(512, 3, activation='softplus'))
    model.add(GlobalMaxPool1D())
    model.add(Dense(units=32, activation='sigmoid'))
    model.add(Dense(units=32, activation='softplus'))
    model.add(Dense(units=16, activation='tanh'))
    model.add(Dense(units=16, activation='softplus'))
    model.add(Dense(units=len(y_train[0]), activation='softplus'))
    # compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    # F1, precision, and recall removed. https://github.com/keras-team/keras/issues/5794
    # fit the model
    history=model.fit(x_train, y_train, validation_split=0.2, epochs=4, verbose=1)
    y_prob = model.predict(x_test, verbose=1)

In [ ]:
# Calculate on validation dataset.
# From probability --> serial coding, e.g., [4, 3, 2, 55] --> categorical, e.g., [[00010...], [001000..]...]
y_train = lb.inverse_transform(np_utils.to_categorical(y_prob.argmax(axis=-1)))
y_train_prob=[s.max() for s in y_prob]
df_val=pd.DataFrame({'pred':y_train, 
                     'true':y_test, 
                     'prob':y_train_prob})
print('Overall ACC:', len(df_val[df_val.pred==df_val.true])/len(df_val))

In [ ]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_true=y_train, y_pred=y_test))

Draft codes saved in `NN_broad_cat.ipynb`.